In [23]:
import sys, os
print("Kernel Python:", sys.executable)

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import torch
# print("PyTorch:", torch.__version__)

from tqdm import tqdm

if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

print("Current location:", os.getcwd())

from esm.models.esmc import ESMC
from esm.sdk.api import ESMProtein, LogitsConfig

Kernel Python: /zhome/c9/0/203261/miniconda3/envs/esm_gpu/bin/python
Current location: /zhome/c9/0/203261/DBLXXX_osaul/DBLXXX_osaul/tmp/ona_drafts


SyntaxError: invalid syntax (<string>, line 1)

In [10]:
# Load metaanalayis data
interaction_df = pd.read_csv("../data/meta_analysis/source_data.csv")
interaction_df = interaction_df[(interaction_df["A_seq"].notna())& (interaction_df["B_seq"].notna())].reset_index(drop=True)
interaction_df = interaction_df[interaction_df["target_chains"] == "[\"B\"]"]
interaction_df = interaction_df[interaction_df["B_seq"].notna()].reset_index(drop=True)
cols_remain = ["binder_id", "target_id", "binder_chain", "target_chains", "binder", "A_seq", "B_seq"]
interaction_df = interaction_df[cols_remain]
# interaction_df = interaction_df.drop(columns=["binder_chain", "target_chains"])

interaction_df

# interaction_df
# interaction_df.target_chains.unique()
# interaction_df.binder_chain.unique()

,binder_id,target_id,binder_chain,target_chains,binder,A_seq,B_seq
0,EHEE_rd4_0110_min_rise1_21_000000015_0001,VirB8,A,"[""B""]",False,LDFIVFAGPEKAIKFYKEMAKRNLEVKIWIDGDWAVVQVK,ANPYISVANIMLQNYVKQREKYNYDTLKEQFTFIKNASTSIVYMQF...
1,Grafting_Motif0040_ems_3hM_3083_0001_0002,FGFR2,A,"[""B""]",False,SEQDETMHRIVRSVIQHAYKHNDEMAEYFAQNAAEIYKEQNKSEEA...,RSPHRPILQAGLPANASTVVGGDVEFVCKVYSDAQPHIQWIKHVPY...
2,Grafting_Motif0042_ems_3hM_148_0001,FGFR2,A,"[""B""]",False,DYKQLKKHATKLLELAKKDPSSKRDLLRTAASYANKVLFEDSDPRA...,RSPHRPILQAGLPANASTVVGGDVEFVCKVYSDAQPHIQWIKHVPY...
3,Grafting_Motif0042_ems_3hM_1661_0001,FGFR2,A,"[""B""]",False,DEKEELERRANRVAFLAIQIQNEEYHRILAELYVQFMKAAENNDTE...,RSPHRPILQAGLPANASTVVGGDVEFVCKVYSDAQPHIQWIKHVPY...
4,Grafting_Motif0042_ems_3hM_257_0001,FGFR2,A,"[""B""]",False,PDNKEKLMSIAVQLILRINEAARSEEQWRYANRAAFAAVEASSGSD...,RSPHRPILQAGLPANASTVVGGDVEFVCKVYSDAQPHIQWIKHVPY...
...,...,...,...,...,...,...,...
3527,il2ra_site1_2b5i_sap_19_mot_HHH_b2_06055_af2_0...,IL2Ra,A,"[""B""]",False,DLRKYAAELVDRLAEKYNLDSDQYNALVRLASELVWQGKSKEEIEK...,ELCDDDPPEIPHATFKAMAYKEGTMLNCECKRGFRRIKSGSLYMLC...
3528,il2ra_site1_2b5i_sap_20_mot_HHH_b1_01934_af2_0...,IL2Ra,A,"[""B""]",False,SKEEIKKEAEELIEELKKKGYNLPLRILEFALKEIEETNSEKYYEQ...,ELCDDDPPEIPHATFKAMAYKEGTMLNCECKRGFRRIKSGSLYMLC...
3529,il2ra_site1_2b5i_sap_23_mot_88fc943612ced423dc...,IL2Ra,A,"[""B""]",False,SPEYKKFLELIKEAEAARKAGDLDKAKELLEKALELAKKMKAKSLI...,ELCDDDPPEIPHATFKAMAYKEGTMLNCECKRGFRRIKSGSLYMLC...
3530,il2ra_site1_2b5i_sap_25_mot_HHH_b2_01943_00000...,IL2Ra,A,"[""B""]",False,DPLLAYKLLKLSQKALEKAYAEDRERAEELLEEAEAALRSLGDEAG...,ELCDDDPPEIPHATFKAMAYKEGTMLNCECKRGFRRIKSGSLYMLC...


In [11]:
interaction_df[interaction_df.target_id.isna() == True]

interaction_df["target_id"].unique()
interaction_df[interaction_df["target_id"] == "EGFR"]["B_seq"].unique()

array(['RKVCNGIGIGEFKDSLSINATNIKHFKNCTSISGDLHILPVAFRGDSFTHTPPLDPQELDILKTVKEITGFLLIQAWPENRTDLHAFENLEIIRGRTKQHGQFSLAVVSLNITSLGLRSLKEISDGDVIISGNKNLCYANTINWKKLFGTSGQKTKIISNRGENSCKATGQVCHALCSPEGCWGPEPRDCV',
       'LEEKKVCQGTSNKLTQLGTFEDHFLSLQRMFNNCEVVLGNLEITYVQRNYDLSFLKTIQEVAGYVLIALNTVERIPLENLQIIRGNMYYENSYALAVLSNYDANKTGLKELPMRNLQEILHGAVRFSNNPALCNVESIQWRDIVSSDFLSNMSMDFQNHLGSCQKCDPSCPNGSCWGAGEENCQKLTKIICAQQCSGRCRGKSPSDCCHNQCAAGCTGPRESDCLVCRKFRDEATCKDTCPPLMLYNPTTYQMDVNPEGKYSFGATCVKKCPRNYVVTDHGSCVRACGADSYEMEEDGVRKCKKCEGPCRKVCNGIGIGEFKDSLSINATNIKHFKNCTSISGDLHILPVAFRGDSFTHTPPLDPQELDILKTVKEITGFLLIQAWPENRTDLHAFENLEIIRGRTKQHGQFSLAVVSLNITSLGLRSLKEISDGDVIISGNKNLCYANTINWKKLFGTSGQKTKIISNRGENSCKATGQVCHALCSPEGCWGPEPRDCVSCRNVSRGRECVDKCNLLEGEPREFVENSECIQCHPECLPQAMNITCTGRGPDNCIQCAHYIDGPHCVKTCPAGVMGENNTLVWKYADAGHVCHLCHPNCTYGCTGPGLEGCPTNGPKIPS',
       'VCQGTSNKLTQLGTFEDHFLSLQRMFNNCEVVLGNLEITYVQRNYDLSFLKTIQEVAGYVLIALNTVERIPLENLQIIRGNMYYENSYALAVLSNYDANKTGLKELPMRNLQEILHGAVRFSNNPALCNVESIQWRDIVSSDFLSNMSMDFQNHLGS'

In [12]:
for i in interaction_df["target_id"].unique():
    x = interaction_df[interaction_df["target_id"] == i]["B_seq"].unique()
    # print(i, len(x), x)

In [13]:
cnt = interaction_df.groupby('target_id').cumcount() + 1
interaction_df['target_binder_ID'] = interaction_df['target_id'].astype(str) + '_' + cnt.astype(str)
interaction_df

,binder_id,target_id,binder_chain,target_chains,binder,A_seq,B_seq,target_binder_ID
0,EHEE_rd4_0110_min_rise1_21_000000015_0001,VirB8,A,"[""B""]",False,LDFIVFAGPEKAIKFYKEMAKRNLEVKIWIDGDWAVVQVK,ANPYISVANIMLQNYVKQREKYNYDTLKEQFTFIKNASTSIVYMQF...,VirB8_1
1,Grafting_Motif0040_ems_3hM_3083_0001_0002,FGFR2,A,"[""B""]",False,SEQDETMHRIVRSVIQHAYKHNDEMAEYFAQNAAEIYKEQNKSEEA...,RSPHRPILQAGLPANASTVVGGDVEFVCKVYSDAQPHIQWIKHVPY...,FGFR2_1
2,Grafting_Motif0042_ems_3hM_148_0001,FGFR2,A,"[""B""]",False,DYKQLKKHATKLLELAKKDPSSKRDLLRTAASYANKVLFEDSDPRA...,RSPHRPILQAGLPANASTVVGGDVEFVCKVYSDAQPHIQWIKHVPY...,FGFR2_2
3,Grafting_Motif0042_ems_3hM_1661_0001,FGFR2,A,"[""B""]",False,DEKEELERRANRVAFLAIQIQNEEYHRILAELYVQFMKAAENNDTE...,RSPHRPILQAGLPANASTVVGGDVEFVCKVYSDAQPHIQWIKHVPY...,FGFR2_3
4,Grafting_Motif0042_ems_3hM_257_0001,FGFR2,A,"[""B""]",False,PDNKEKLMSIAVQLILRINEAARSEEQWRYANRAAFAAVEASSGSD...,RSPHRPILQAGLPANASTVVGGDVEFVCKVYSDAQPHIQWIKHVPY...,FGFR2_4
...,...,...,...,...,...,...,...,...
3527,il2ra_site1_2b5i_sap_19_mot_HHH_b2_06055_af2_0...,IL2Ra,A,"[""B""]",False,DLRKYAAELVDRLAEKYNLDSDQYNALVRLASELVWQGKSKEEIEK...,ELCDDDPPEIPHATFKAMAYKEGTMLNCECKRGFRRIKSGSLYMLC...,IL2Ra_62
3528,il2ra_site1_2b5i_sap_20_mot_HHH_b1_01934_af2_0...,IL2Ra,A,"[""B""]",False,SKEEIKKEAEELIEELKKKGYNLPLRILEFALKEIEETNSEKYYEQ...,ELCDDDPPEIPHATFKAMAYKEGTMLNCECKRGFRRIKSGSLYMLC...,IL2Ra_63
3529,il2ra_site1_2b5i_sap_23_mot_88fc943612ced423dc...,IL2Ra,A,"[""B""]",False,SPEYKKFLELIKEAEAARKAGDLDKAKELLEKALELAKKMKAKSLI...,ELCDDDPPEIPHATFKAMAYKEGTMLNCECKRGFRRIKSGSLYMLC...,IL2Ra_64
3530,il2ra_site1_2b5i_sap_25_mot_HHH_b2_01943_00000...,IL2Ra,A,"[""B""]",False,DPLLAYKLLKLSQKALEKAYAEDRERAEELLEEAEAALRSLGDEAG...,ELCDDDPPEIPHATFKAMAYKEGTMLNCECKRGFRRIKSGSLYMLC...,IL2Ra_65


In [16]:
dfu = interaction_df.drop_duplicates(["target_id", "B_seq"])
dfu[dfu.target_id == "EGFR"].B_seq.tolist()
targets_Dict, target_tracking = {}, {target:int(2) for target in dfu.target_id.unique()}

for idx, target_id in enumerate(dfu.target_id):
    if target_id not in targets_Dict.keys():
        targets_Dict[target_id] = dfu.iloc[idx]["B_seq"]
        # print(target_id)
    else:
        target_id_new = f"{target_id}_{target_tracking[target_id]}"
        targets_Dict[target_id_new] = dfu.iloc[idx]["B_seq"]
        count = target_tracking[target_id]
        target_tracking[target_id] = count + 1
        # print(target_id_new)

interaction_df['target_id_mod'] = interaction_df['B_seq'].map({b:a for a, b in targets_Dict.items()})
interaction_df['B_seq'].map({b:a for a, b in targets_Dict.items()})
targets_Dict.keys()

dict_keys(['VirB8', 'FGFR2', 'IL7Ra', 'InsulinR', 'EGFR', 'SARS_CoV2_RBD', 'Pdl1', 'EGFR_2', 'TrkA', 'IL10Ra', 'LTK', 'Mdm2', 'EGFR_3', 'sntx', 'sntx_2', 'IL2Ra'])

In [17]:
interaction_df[interaction_df.target_id_mod.isna() == True]

,binder_id,target_id,binder_chain,target_chains,binder,A_seq,B_seq,target_binder_ID,target_id_mod


In [18]:
binders_Dict = {}
for idx, binder_id in enumerate(interaction_df.target_binder_ID):
    if binder_id not in binders_Dict.keys():
        binders_Dict[binder_id] = interaction_df.iloc[idx]["A_seq"]
    else:
        print("Something is off..")
        print(binder_id, interaction_df.iloc[idx]["A_seq"])
# binders_Dict

In [19]:
# device = "cuda" if torch.cuda.is_available() else "cpu"
# Using CPU for downloading/ loading the model
# device = "cpu"
client = ESMC.from_pretrained("esmc_600m", device=torch.device("cpu")).to("cpu")

# if device == "cuda":
#     client = client.to("cuda").half()              # FP16 fallback
# else:
#     client = client.to("cpu")

# protein = ESMProtein(sequence="AAAAAAA")
# pt = client.encode(protein)                 # usually returns a CPU tensor
# pt = pt.to(next(client.parameters()).device)

# out = client.logits(pt, LogitsConfig(sequence=True, return_embeddings=True))

# Moving model to cuda
client_gpu = client.to("cuda")

NameError: name 'ESMC' is not defined

In [ ]:
def calculate_ESM_pr_res_embeddings(sequence):
    protein = ESMProtein(sequence=sequence)
    protein_tensor = client_gpu.encode(protein)
    logits_output = client_gpu.logits(
    protein_tensor, LogitsConfig(sequence=True, return_embeddings=True)
    )
    return logits_output.embeddings.detach().cpu().numpy()

# Test run
sequence = "AAAAAAA"
embedding = calculate_ESM_pr_res_embeddings(sequence)
embedding.shape

In [ ]:
# # where to save
# path_to_output_embeddings = "/zhome/c9/0/203261/DBLXXX_osaul/DBLXXX_osaul/tmp/data/meta_analysis/targets_embeddings"
# os.makedirs(path_to_output_embeddings, exist_ok=True)

# # helper: convert torch tensor to numpy
# def to_numpy(x):
#     try:
#         return x.detach().cpu().numpy()
#     except AttributeError:
#         return np.asarray(x)

# # iterate targets
# for name, seqs in tqdm(targets_Dict.items(), total=len(targets_Dict), desc="Embedding targets"):
#     emb = calculate_ESM_pr_res_embeddings(seqs)
#     emb_np = to_numpy(emb)
#     out_path = os.path.join(path_to_output_embeddings, f"{name}.npy")
#     np.save(out_path, emb_np)
#     print(f"Target {name} embedded and saved to {out_path}")

# # Count files in the 
# # path_to_output_embeddings = "/zhome/c9/0/203261/DBLXXX_osaul/DBLXXX_osaul/tmp/data/meta_analysis/targets_embeddings"
# if len(os.listdir(path_to_output_embeddings)) == len(targets_Dict.keys()):
#     print("All targets!")

In [ ]:
# # where to save
# path_to_output_embeddings = "/zhome/c9/0/203261/DBLXXX_osaul/DBLXXX_osaul/tmp/data/meta_analysis/binders_embeddings"
# os.makedirs(path_to_output_embeddings, exist_ok=True)

# # iterate targets
# for name, seqs in tqdm(binders_Dict.items(), total=len(binders_Dict), desc="Embedding targets"):
#     emb = calculate_ESM_pr_res_embeddings(seqs)
#     emb_np = to_numpy(emb)
#     out_path = os.path.join(path_to_output_embeddings, f"{name}.npy")
#     np.save(out_path, emb_np)
#     print(f"Binder {name} embedded and saved to {out_path}")

# # Count files in the 
# # path_to_output_embeddings = "/zhome/c9/0/203261/DBLXXX_osaul/DBLXXX_osaul/tmp/data/meta_analysis/binders_embeddings"
# if len(os.listdir(path_to_output_embeddings)) == len(binders_Dict.keys()):
#     print("All binders!")

In [ ]:
path_to_output_embeddings = "/zhome/c9/0/203261/DBLXXX_osaul/DBLXXX_osaul/tmp/data/meta_analysis/binders_embeddings"
name = "EGFR_18"
path_to_output_file = os.path.join(path_to_output_embeddings,f"{name}.npy")
random_embedding = np.load(path_to_output_file)
random_embedding.shape

In [ ]:
print(binders_Dict["EGFR_18"])
print(len(binders_Dict["EGFR_18"]))

In [ ]:
interaction_df["A_seq"].apply(lambda x: len(x)).max()
interaction_df["B_seq"].apply(lambda x: len(x)).max()

In [ ]:
path_to_output_embeddings = "/zhome/c9/0/203261/DBLXXX_osaul/DBLXXX_osaul/tmp/data/meta_analysis/binders_embeddings"
name = "EGFR_18"
path_to_output_file = os.path.join(path_to_output_embeddings,f"{name}.npy")
random_embedding = np.load(path_to_output_file)
random_embedding.shape

In [ ]:
seq = binders_Dict["EGFR_18"]
len(seq)

In [ ]:
# Save modified interaction_df
interaction_df.to_csv("/zhome/c9/0/203261/DBLXXX_osaul/DBLXXX_osaul/tmp/data/meta_analysis/interaction_df_metaanal.csv")

In [ ]:
os.listdir("/zhome/c9/0/203261/DBLXXX_osaul/DBLXXX_osaul/tmp/data/meta_analysis/")